# Filters
Coba Filters modify Environments. An environmental Filter is any class with the following interface:

```python
class EnvironmentalFilter:
    def filter(self, interactions:Iterable[Interaction])->Iterable[Interaction]:
        ...
```
Where **[Interaction](Interactions.ipynb)** is a Python `dict` with specific entires. Most commonly it is, 
```python
{'context':..., 'actions':..., 'rewards':...}.
```

# Preliminaries

For a complete list of the environment filters provided by Coba out-of-the-box see...

For more information regarding the Environments API used below see [here](Environments.ipynb).

# A Simple Example

Below we create and apply a Filter to set all interaction contexts to 1. Applying a filter to an Environments object is done via the `filter` method.

In [1]:
import coba as cb

class SetContextToOne:
    def filter(self,interactions):
        for old in interactions:
            new = old.copy() #we copy to make sure we don't contaminate other filters
            new['context'] = 1 #set the context to 1
            yield new #then yield the updated interaction

env = cb.Environments.from_linear_synthetic(100, n_actions=2, n_context_features=1, n_action_features=1)
one = env.filter(SetContextToOne())

print("------------------------------------------------")
print("First interaction in unfiltered environment")
print("------------------------------------------------")
print("  ",end="")
print(cb.minimize(next(env[0].read()),2))

print()
print("------------------------------------------------")
print("First interaction after applying SetContextToOne")
print("------------------------------------------------")
print("  ",end="")
print(cb.minimize(next(one[0].read()),2))

------------------------------------------------
First interaction in unfiltered environment
------------------------------------------------
  {'context': [-0.75], 'actions': [[-0.59], [0.95]], 'rewards': DiscreteReward([[[-0.59205], [0.94641]], [0.17759, 1.02217]])}

------------------------------------------------
First interaction after applying SetContextToOne
------------------------------------------------
  {'context': 1, 'actions': [[-0.59], [0.95]], 'rewards': DiscreteReward([[[-0.59205], [0.94641]], [0.17759, 1.02217]])}


# Adding Descriptive Parameters

We can also provide parameters to describe a Filter. In the context of an Experiment these params go to `Result.environments`.

In [9]:
import coba as cb

class ContextToN:
    def __init__(self,N):
        self._N = N

    @property
    def params(self):
        return {'N':self._N}

    def filter(self,interactions):
        for old in interactions:
            new = old.copy() #we copy to make sure we don't pollute other filters
            new['context'] = self._N #set the context to 1
            yield new #then yield the updated interaction

env = cb.Environments.from_linear_synthetic(100, n_actions=2, n_context_features=1, n_action_features=1)
two = env.filter(ContextToN(2))

print("----------------------------------------------")
print("Descriptive parameters before filtering")
print("----------------------------------------------")
print("  ",end="")
print(env[0].params)

print()
print("----------------------------------------------")
print("Descriptive parameters after filtering")
print("----------------------------------------------")
print("  ",end="")
print(two[0].params)

----------------------------------------------
Descriptive parameters before filtering
----------------------------------------------
  {'env_type': 'LinearSynthetic', 'reward_features': ['a', 'xa'], 'n_coeff': 5, 'seed': 1}

----------------------------------------------
Descriptive parameters after filtering
----------------------------------------------
  {'env_type': 'LinearSynthetic', 'reward_features': ['a', 'xa'], 'n_coeff': 5, 'seed': 1, 'N': 2}


# Manually Applying Filters

It is possible to manually apply filters to environments though it is strongly recommended to use the Environments API as shown above.

Below is a quick example showing how a filter can be manually applied via the Pipes module.

In [11]:
import coba as cb

class ContextToN:
    def __init__(self,N):
        self._N = N

    @property
    def params(self):
        return {'N':self._N}

    def filter(self,interactions):
        for old in interactions:
            new = old.copy() #we copy to make sure we don't pollute other filters
            new['context'] = self._N #set the context to 1
            yield new #then yield the updated interaction

env = cb.Environments.from_linear_synthetic(100, n_actions=2, n_context_features=1, n_action_features=1)[0]
two = cb.pipes.Pipes.join(env,ContextToN(2),ContextToN(1)) # Any number of filters can be applied

print("----------------------------------------------")
print("Descriptive parameters before filtering")
print("----------------------------------------------")
print("  ",end="")
print(env.params)

print()
print("----------------------------------------------")
print("Descriptive parameters after filtering")
print("----------------------------------------------")
print("  ",end="")
print(two.params)

----------------------------------------------
Descriptive parameters before filtering
----------------------------------------------
  {'env_type': 'LinearSynthetic', 'reward_features': ['a', 'xa'], 'n_coeff': 5, 'seed': 1}

----------------------------------------------
Descriptive parameters after filtering
----------------------------------------------
  {'env_type': 'LinearSynthetic', 'reward_features': ['a', 'xa'], 'n_coeff': 5, 'seed': 1, 'N1': 2, 'N2': 1}
